In [3]:
import json
import pandas as pd

from tqdm import tqdm
from datetime import datetime, timedelta

In [86]:
line_count = len(open("data/review.json").readlines())
user_ids, business_ids, stars, dates = [], [], [], []
count = 0;
with open("data/review.json") as f:
    for line in tqdm(f, total=line_count):
        blob = json.loads(line)
        user_ids += [blob["user_id"]]
        business_ids += [blob["business_id"]]
        stars += [blob["stars"]]
        dates += [blob["date"]]
    
ratings = pd.DataFrame({"user_id": user_ids, 
                        "business_id": business_ids, 
                        "rating": stars, "date": dates})

user_counts = ratings["user_id"].value_counts()
active_users = user_counts.loc[user_counts >= 5].index.tolist()        


 25%|██▍       | 1653930/6685900 [00:12<00:39, 126813.58it/s]


 50%|█████     | 3350035/6685900 [00:25<00:25, 131864.87it/s]


 75%|███████▍  | 4981018/6685900 [00:38<00:12, 134130.43it/s]


 99%|█████████▉| 6630296/6685900 [00:51<00:00, 122455.73it/s]


100%|██████████| 6685900/6685900 [00:51<00:00, 128839.77it/s]

In [4]:
line_count = len(open('data/business.json').readlines())
business_ids, city, state, stars, review_count, categories, hours = [], [], [], [], [], [], []

with open('data/business.json') as f:
    for line in tqdm(f, total=line_count):
        blob = json.loads(line)
        business_ids += [blob["business_id"]]
        city += [blob["city"]]
        state += [blob["state"]]
        stars += [blob["stars"]]
        review_count += [blob["review_count"]]
        if blob["categories"] is None:
            categories += ['None']
        else:
            categories += [blob["categories"].split(', ')]
        hours += [blob["hours"]]
        
business = pd.DataFrame({"business_id": business_ids, 
                         "stars": stars, 
                         "review_count": review_count, 
                         "categories": categories, 
                         "hours": hours})

100%|██████████| 192609/192609 [00:03<00:00, 63253.05it/s]


In [48]:
line_count = len(open('data/checkin.json').readlines())
business_ids, dates = [], []

with open('data/checkin.json') as f:
    for line in tqdm(f, total=line_count):
        blob = json.loads(line)
        business_ids += [blob["business_id"]]
        dates += [blob["date"]]

checkin = pd.DataFrame({"business_id": business_ids, 
                        "date": dates})

100%|██████████| 161950/161950 [00:01<00:00, 125290.01it/s]


In [84]:
def add_item_features(business, checkin):
    all_categories = []
    business['hours'] = business['hours'].apply(lambda x: ' '.join(' '.join(str(x).split(', ')).split(': ')).split())
    
    for i in tqdm(range(len(business))):
        all_categories.extend(business.loc[i, 'categories'])
        
        if business.loc[i, 'hours'] == ['None'] or []:
            business.loc[i, 'total_hours'] = 0
        else:
            open_hours = business.loc[i, 'hours'][1::2]
            open_hours[-1] = open_hours[-1][:-1]
            open_hours = [x[1:len(x)-1].split('-') for x in open_hours]
            total_hours = 0

            for j in range(len(open_hours)):
                delta = datetime.strptime(open_hours[j][1], '%H:%M') - datetime.strptime(open_hours[j][0], '%H:%M')
                if delta <= timedelta(days = -1, hours = 23):
                    delta = delta - timedelta(days=-1)

                delta = delta.seconds//3600
                total_hours = total_hours + delta
            business.loc[i, 'total_hours'] = total_hours
    
    all_categories = list(set(all_categories))
    category_features = pd.DataFrame(index = range(0, len(business)), columns = all_categories)
    
    for i in tqdm(range(len(business))):
        for j in range(len(business.loc[i, 'categories'])):
            category_features.loc[i, business.loc[i, 'categories'][j]] = 1
    
    checkin['total_checkins'] = checkin['date'].apply(lambda x: len(str(x).split(', ')))
    checkin['first_checkin'] = checkin['date'].apply(lambda x: min(str(x).split(', ')))
    checkin['last_checkin'] = checkin['date'].apply(lambda x: max(str(x).split(', ')))
    
    for i in tqdm(range(len(checkin))):
        last_checkin = datetime.strptime(str(checkin.loc[i, 'last_checkin']), '%Y-%m-%d %H:%M:%S').date()
        first_checkin = datetime.strptime(str(checkin.loc[i, 'first_checkin']), '%Y-%m-%d %H:%M:%S').date()
        
        checkin['age_of_business'] = (last_checkin - first_checkin).days
            
    # Pending - To join business and checkin
    return business

#### Direct Features
1. useful, funny, cool total votes - normalized
2. total fans, total friends in user's network
3. number of years user is elite, or 1-hot encoding of each year
4. yelping since - number of days
5. total number of reviews
6. compliment count on tips - will need to join tips
7. average rating of all reviews given / user bias

Business
1. Average Open Hours in a Week, Day of the Week
2. is_weekend, is_weekday open
3. City/State - Location 
4. Total Ratings, Average Ratings
5. user diversity - to be normalized
6. frequency of checkins
7. age of business using checkins

#### Derived Features

In [85]:
business, checkin = add_item_features(business, checkin)

  0%|          | 2/192609 [01:40<2060:53:45, 38.52s/it]

KeyboardInterrupt: 

In [ ]:
def user_location_pref(ratings, )

In [ ]:
def user_review_score(ratings, ) #based on compliments received on his review - funny, cool, etc.

In [ ]:
def user_days_since_last_review(ratings, ) #can be a useful feature, maybe